# Configuration

NOTES: The warnings after the import are referred to the fact that Tensorflow 2.x versions are built to directly look for a GPU in the system. The warning can be forgot if you are not going to use the GPU. 

NOTE: Differently from experiments 1-4, here we have the 'mean' and 'std' of mfccs in **compute_mfccs** function

In [1]:
!source myenv/bin/activate

In [2]:
# samples in 5 seconds of audio, 16 KHz sample rate 
LENGTH_CHOSEN =  80000

In [3]:
import os
import librosa
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set_style('whitegrid')
import IPython.display as ipd
import librosa.display
import numpy as np
import pickle
import scipy
import ipywidgets
import math
from time import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold


from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, AveragePooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# from livelossplot import PlotLossesKeras
tf.config.list_physical_devices('GPU')

2021-09-27 19:14:19.873805: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-27 19:14:19.873850: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-27 19:14:21.997703: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-27 19:14:21.998510: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-27 19:14:22.094598: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-27 19:14:22.094628: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /pr

[]

In [4]:
# import SVM
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC 

In [5]:
# set reproducibility 
seed = 7
np.random.seed(seed)

# Utils

In [6]:
def load_files(df):
    X = []
    for i in tqdm(df['path']): 
        X.append(librosa.load(i, res_type='kaiser_fast', sr=16000))
    return X

def extract_samples(X): 
    samples = []
    for ind,i in enumerate(X):
        samples.append(i[0])
    return samples 

def extract_labels(df): 
    labels = df['emotion_label'].copy()
    return labels 

def compute_lengths(samples): 
    lengths = [len(x) for x in samples]
    return lengths 

def check_outliers(lengths):
    # outliers
    lengths = np.array(lengths)
    print((lengths > 300000).sum())
    new_lengths = lengths[lengths < 300000]
    return new_lengths 

def compute_mean_length(lengths): 
    return lengths.mean()

def cut_and_pad(samples, labels, length_chosen = LENGTH_CHOSEN): 
    X_new = []
    y_new = []
    count = 0 
    for ind,i in enumerate(samples):
        if i.shape[0] < 300000:
            if i.shape[0] > length_chosen:
                new = i[:length_chosen]
                X_new.append(new)
            elif i.shape[0] < length_chosen:
                new = np.pad(i,math.ceil((length_chosen-i.shape[0])/2), mode='median')
                X_new.append(new)
            else:
                X_new.append(i)
            y_new.append(labels[count])
        count+=1
    
    return X_new, y_new

def compute_energy(samples): 
    energy_mean = []
    for i in tqdm(samples):
        energy = librosa.feature.rms(i)
        energy = energy.T 
        energy = np.array(energy)
        energy_mean.append(np.mean(energy, axis=0)) 
    return energy_mean

def compute_mfccs(samples, n_mfcc): 
    mfccs = []
    for i in tqdm(samples):
        mfcc = librosa.feature.mfcc(y=i, sr=16000, n_mfcc=n_mfcc)
        mfcc = mfcc.T
        mfcc = np.array(mfcc)
        #mfccs.append(mfcc[:, 1:]) # get rid of the first component 
        mfccs.append(np.mean(mfcc[:, 1:], axis = 0))
    mfccs = np.array(mfccs)
    return mfccs

def feature_extractor(df_train, df_val, df_test, n_mfcc): 
    load_train = load_files(df_train)
    samples_train = extract_samples(load_train)
    labels_train = extract_labels(df_train)
    samples_train, labels_train = cut_and_pad(samples_train, labels_train)
    samples_train = np.array(samples_train)
    labels_train = np.array(labels_train)
    mfccs_train = compute_mfccs(samples_train, n_mfcc = n_mfcc)
    # energy 
    energy_train = compute_energy(samples_train) 
    features_train = []
    for i in range(len(mfccs_train)): 
        if len(mfccs_train) == len(energy_train): 
            conc = np.concatenate((mfccs_train[i], energy_train[i]), axis = None)
            features_train.append(conc)

    
    
  
    load_val = load_files(df_val)
    samples_val = extract_samples(load_val)
    labels_val = extract_labels(df_val)
    samples_val, labels_val = cut_and_pad(samples_val, labels_val)
    samples_val = np.array(samples_val)
    labels_val = np.array(labels_val)
    mfccs_val = compute_mfccs(samples_val, n_mfcc = n_mfcc)
    # energy 
    energy_val = compute_energy(samples_val) 
    features_val = []
    for i in range(len(mfccs_val)): 
        if len(mfccs_val) == len(energy_val): 
            conc = np.concatenate((mfccs_val[i], energy_val[i]), axis = None)
            features_val.append(conc)
    
    
    
    
    load_test = load_files(df_test)
    samples_test = extract_samples(load_test)
    labels_test = extract_labels(df_test)
    samples_test, labels_test = cut_and_pad(samples_test, labels_test)
    samples_test = np.array(samples_test)
    labels_test = np.array(labels_test)
    mfccs_test = compute_mfccs(samples_test, n_mfcc = n_mfcc)
    # energy 
    energy_test = compute_energy(samples_test) 
    features_test=[]
    for i in range(len(mfccs_test)): 
        if len(mfccs_test) == len(energy_test): 
            conc = np.concatenate((mfccs_test[i], energy_test[i]), axis = None)
            features_test.append(conc)
    

    return np.array(features_train), labels_train,  np.array(features_val), labels_val, np.array(features_test), labels_test
    

def feature_extractor_tess(df_train,  df_test, n_mfcc): 
    # we do not have the validation set here 
    load_train = load_files(df_train)
    samples_train = extract_samples(load_train)
    labels_train = extract_labels(df_train)
    samples_train, labels_train = cut_and_pad(samples_train, labels_train)
    samples_train = np.array(samples_train)
    labels_train = np.array(labels_train)
    mfccs_train = compute_mfccs(samples_train, n_mfcc = n_mfcc)
    # energy 
    energy_train = compute_energy(samples_train) 
    features_train = []
    for i in range(len(mfccs_train)): 
        if len(mfccs_train) == len(energy_train): 
            conc = np.concatenate((mfccs_train[i], energy_train[i]), axis = None)
            features_train.append(conc)

    
    
    load_test = load_files(df_test)
    samples_test = extract_samples(load_test)
    labels_test = extract_labels(df_test)
    samples_test, labels_test = cut_and_pad(samples_test, labels_test)
    samples_test = np.array(samples_test)
    labels_test = np.array(labels_test)
    mfccs_test = compute_mfccs(samples_test, n_mfcc = n_mfcc)
    # energy 
    energy_test = compute_energy(samples_test) 
    features_test=[]
    for i in range(len(mfccs_test)): 
        if len(mfccs_test) == len(energy_test): 
            conc = np.concatenate((mfccs_test[i], energy_test[i]), axis = None)
            features_test.append(conc)
    
    
    return np.array(features_train), labels_train, np.array(features_test), labels_test
    
def encode_labels(labels_train, labels_val, labels_test): 
    
    emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
    y_train = pd.Series(labels_train).replace(emotion_enc)
  
    y_test = pd.Series(labels_test).map(emotion_enc)
    y_val = pd.Series(labels_val).map(emotion_enc)
    return y_train, y_val, y_test 


def encode_labels_tess(labels_train, labels_test): 
    
    emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
    y_train = pd.Series(labels_train).replace(emotion_enc)
  
    y_test = pd.Series(labels_test).map(emotion_enc)
    return y_train, y_test
    
def standard_scaling(X_train, X_val, X_test): 
  
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
    return X_train, X_val, X_test 
    
def standard_scaling_tess(X_train, X_test): 
  
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    return X_train, X_test   
    


# Compute dataframes for datasets and split in Train, Val, Test 

In [7]:
main_path = '/media/helemanc/OS/Users/i2CAT/Desktop/Datasets SER/'
TESS = os.path.join(main_path, "tess/TESS Toronto emotional speech set data/") 
RAV = os.path.join(main_path, "ravdess-emotional-speech-audio/audio_speech_actors_01-24")
SAVEE = os.path.join(main_path, "savee/ALL/")
CREMA = os.path.join(main_path, "creamd/AudioWAV/")

## RADVESS

In [8]:
lst = []
emotion = []
voc_channel = []
full_path = []
modality = []
intensity = []
actors = []
phrase =[]

for root, dirs, files in tqdm(os.walk(RAV)):
    for file in files:
        try:
            #Load librosa array, obtain mfcss, store the file and the mfcss information in a new array
            # X, sample_rate = librosa.load(os.path.join(root,file), res_type='kaiser_fast')
            # mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
            # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
            # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
           
            modal = int(file[1:2])
            vchan = int(file[4:5])
            lab = int(file[7:8])
            ints = int(file[10:11])
            phr = int(file[13:14])
            act = int(file[18:20])
            # arr = mfccs, lab
            # lst.append(arr)
            
            modality.append(modal)
            voc_channel.append(vchan)
            emotion.append(lab) #only labels
            intensity.append(ints)
            phrase.append(phr)
            actors.append(act)
            
            full_path.append((root, file)) # only files
          # If the file is not valid, skip it
        except ValueError:
            continue

25it [00:00, 859.27it/s]


In [9]:
# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# merge neutral and calm
emotions_list = ['neutral', 'neutral', 'happy', 'sadness', 'angry', 'fear', 'disgust', 'surprise']
emotion_dict = {em[0]+1:em[1] for em in enumerate(emotions_list)}

df = pd.DataFrame([emotion, voc_channel, modality, intensity, actors, actors,phrase, full_path]).T
df.columns = ['emotion', 'voc_channel', 'modality', 'intensity', 'actors', 'gender', 'phrase', 'path']
df['emotion'] = df['emotion'].map(emotion_dict)
df['voc_channel'] = df['voc_channel'].map({1: 'speech', 2:'song'})
df['modality'] = df['modality'].map({1: 'full AV', 2:'video only', 3:'audio only'})
df['intensity'] = df['intensity'].map({1: 'normal', 2:'strong'})
df['actors'] = df['actors']
df['gender'] = df['actors'].apply(lambda x: 'female' if x%2 == 0 else 'male')
df['phrase'] = df['phrase'].map({1: 'Kids are talking by the door', 2:'Dogs are sitting by the door'})
df['path'] = df['path'].apply(lambda x: x[0] + '/' + x[1])

In [10]:
# remove files with noise to apply the same noise to all files for data augmentation 
df = df[~df.path.str.contains('noise')]

In [11]:
df.head()

,emotion,voc_channel,modality,intensity,actors,gender,phrase,path
0,disgust,speech,audio only,normal,1,male,Dogs are sitting by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,disgust,speech,audio only,strong,1,male,Kids are talking by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,disgust,speech,audio only,strong,1,male,Kids are talking by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
6,disgust,speech,audio only,strong,1,male,Dogs are sitting by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
8,disgust,speech,audio only,strong,1,male,Dogs are sitting by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [12]:
# only speech
RAV_df = df
RAV_df = RAV_df.loc[RAV_df.voc_channel == 'speech']

In [13]:
RAV_df.insert(0, "emotion_label", RAV_df.emotion, True)

In [14]:
RAV_df = RAV_df.drop(['emotion', 'voc_channel', 'modality', 'intensity', 'phrase'], 1)

In [15]:
RAV_df

,emotion_label,actors,gender,path
0,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
6,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
8,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
...,...,...,...,...
2871,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2873,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2875,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2877,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [16]:
RAV_train = []
RAV_val = []
RAV_test = []

In [17]:
for index, row in RAV_df.iterrows():
    if row['actors'] in range(1,21): 
        RAV_train.append(row) 
    elif row['actors'] in range(21,23): 
        RAV_val.append(row)
    elif row['actors'] in range(23,25): 
        RAV_test.append(row)
len(RAV_train), len(RAV_val), len(RAV_test)

(1200, 120, 120)

In [18]:
RAV_train = pd.DataFrame(RAV_train)
RAV_val = pd.DataFrame(RAV_val)
RAV_test = pd.DataFrame(RAV_test)

In [19]:
RAV_train = RAV_train.drop(['actors'], 1)
RAV_val = RAV_val.drop(['actors'], 1)
RAV_test = RAV_test.drop(['actors'], 1)

In [20]:
RAV_train.reset_index(drop=True, inplace = True) 
RAV_val.reset_index(drop=True, inplace = True) 
RAV_test.reset_index(drop=True, inplace = True ) 

## SAVEE

In [21]:
# Get the data location for SAVEE
dir_list = os.listdir(SAVEE)

# parse the filename to get the emotions
emotion=[]
path = []
actors = []
gender = []
for i in dir_list:
    actors.append(i[:2])
    if i[-8:-6]=='_a':
        emotion.append('angry')
        gender.append('male')
    elif i[-8:-6]=='_d':
        emotion.append('disgust')
        gender.append('male')
    elif i[-8:-6]=='_f':
        emotion.append('fear')
        gender.append('male')
    elif i[-8:-6]=='_h':
        emotion.append('happy')
        gender.append('male')
    elif i[-8:-6]=='_n':
        emotion.append('neutral')
        gender.append('male')
    elif i[-8:-6]=='sa':
        emotion.append('sadness')
        gender.append('male')
    elif i[-8:-6]=='su':
        emotion.append('surprise')
        gender.append('male') 
    else:
        emotion.append('Unknown') 
    path.append(SAVEE + i)
    
# Now check out the label count distribution 
SAVEE_df = pd.DataFrame(emotion, columns = ['emotion_label'])
                      
SAVEE_df = pd.concat([SAVEE_df,
                      pd.DataFrame(actors, columns = ['actors']),
                      pd.DataFrame(gender, columns = ['gender']), 
                      pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_df.emotion_label.value_counts()

neutral     120
sadness      60
surprise     60
happy        60
disgust      60
fear         60
angry        60
Name: emotion_label, dtype: int64

In [22]:
SAVEE_df.head()

,emotion_label,actors,gender,path
0,neutral,DC,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
1,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
3,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [23]:
SAVEE_train = []
SAVEE_val = []
SAVEE_test = []

In [24]:
#DC, JE, JK, KL
for index, row in SAVEE_df.iterrows(): 
    if row['actors'] == 'DC' or row ['actors'] == 'JE':
        SAVEE_train.append(row)
    elif row['actors'] == 'JK': 
        SAVEE_val.append(row)
    else: 
        SAVEE_test.append(row)
len(SAVEE_train), len(SAVEE_val), len(SAVEE_test)

(240, 120, 120)

In [25]:
SAVEE_train = pd.DataFrame(SAVEE_train)
SAVEE_val = pd.DataFrame(SAVEE_val)
SAVEE_test = pd.DataFrame(SAVEE_test)

In [26]:
SAVEE_train = SAVEE_train.drop(['actors'], 1)
SAVEE_val = SAVEE_val.drop(['actors'], 1)
SAVEE_test = SAVEE_test.drop(['actors'], 1)

In [27]:
SAVEE_train = SAVEE_train.reset_index(drop=True) 
SAVEE_val = SAVEE_val.reset_index(drop=True) 
SAVEE_test = SAVEE_test.reset_index(drop=True) 

## TESS

In [28]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

path = []
emotion = []
gender = []
actors = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        if i == 'OAF_angry':
            emotion.append('angry')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_angry': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_disgust' :
            emotion.append('disgust')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_disgust': 
            emotion.append('disgust')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_Fear':
            emotion.append('fear')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_fear': 
            emotion.append('fear')
            gender.append('female')
            actors.append('YAF') 
            
            
        elif i == 'OAF_happy' :
            emotion.append('happy')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_happy': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_neutral':
            emotion.append('neutral')
            gender.append('female')
            actors.append('OAF')   
        elif i == 'YAF_neutral': 
            emotion.append('neutral')
            gender.append('female')
            actors.append('YAF')      
            
                
        elif i == 'OAF_Pleasant_surprise':
            emotion.append('surprise')
            gender.append('female')
            actors.append('OAF')
        
        elif i == 'YAF_pleasant_surprised': 
            emotion.append('surprise')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_Sad':
            emotion.append('sadness')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_sad': 
            emotion.append('sadness')
            gender.append('female')
            actors.append('YAF')            
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['emotion_label'])
TESS_df = pd.concat([TESS_df, pd.DataFrame(gender, columns = ['gender']), 
                     pd.DataFrame(actors, columns= ['actors']),
                     pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.emotion_label.value_counts()

angry       1200
fear         800
surprise     800
sadness      800
disgust      800
neutral      800
happy        400
Name: emotion_label, dtype: int64

In [29]:
TESS_df= TESS_df[~TESS_df.path.str.contains('noise')]

In [30]:
TESS_train = []
TESS_test = []

In [31]:
for index, row in TESS_df.iterrows(): 
    if row['actors'] == 'YAF': 
        TESS_train.append(row)
    else: 
        TESS_test.append(row)
len(TESS_train), len(TESS_test)

(1400, 1400)

In [32]:
TESS_train = pd.DataFrame(TESS_train)
TESS_test = pd.DataFrame(TESS_test)

In [33]:
TESS_train = TESS_train.reset_index(drop=True) 
TESS_test  = TESS_test.reset_index(drop=True) 

## CREMA-D

In [34]:
males = [1,
5,
11,
14,
15,
16,
17,
19,
22,
23,
26,
27,
31,
32,
33,
34,
35,
36,
38,
39,
41,
42,
44,
45,
48,
50,
51,
57,
59, 
62, 
64,
65, 
66,
67,
68,
69,
70,
71,
77, 
80, 
81, 
83, 
85, 
86, 
87,
88, 
90]

In [35]:
females = [ 2,
3,
4,
6,
7,
8,
9,
10,
12,
13,
18,
20,
21,
24,
25,
28,
29,
30,
37,
40,
43,
46,
47,
49,
52,
53,
54,
55,
56, 
58, 
60,
61,
63,
72, 
73, 
74, 
75, 
76, 
78, 
79, 
82, 
84, 
89, 
91]

In [36]:
crema_directory_list = os.listdir(CREMA)

file_emotion = []
file_path = []
actors = []
gender = []




for file in crema_directory_list:

    # storing file emotions
    part=file.split('_')
    
    # use only high intensity files
    if "HI" in part[3] :
        actor = part[0][2:]
        actors.append(actor)
        if int(actor) in males:
            gender.append('male')
        else: 
            gender.append('female')
    
        # storing file paths
        file_path.append(CREMA + file)
        if part[2] == 'SAD':
            file_emotion.append('sadness')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        else:
            file_emotion.append('Unknown')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['emotion_label'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['path'])
actors_df = pd.DataFrame(actors, columns=['actors'])
gender_df = pd.DataFrame(gender, columns=['gender'])                      
Crema_df = pd.concat([emotion_df, actors_df, gender_df, path_df], axis=1)
Crema_df.head()

,emotion_label,actors,gender,path
0,happy,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
1,sadness,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,angry,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
3,disgust,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,fear,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [37]:
Crema_df.shape

(455, 4)

In [38]:
actor_files = {}

for index, row in Crema_df.iterrows():
    actor = row['actors']
    if actor not in actor_files.keys(): 
        actor_files[actor] = 1
    else: 
        actor_files[actor]+=1

In [39]:
actor_files

{'91': 5,
 '90': 5,
 '89': 5,
 '88': 5,
 '87': 5,
 '86': 5,
 '85': 5,
 '84': 5,
 '83': 5,
 '82': 5,
 '81': 5,
 '80': 5,
 '79': 5,
 '78': 5,
 '77': 5,
 '76': 5,
 '75': 5,
 '74': 5,
 '73': 5,
 '72': 5,
 '71': 5,
 '70': 5,
 '69': 5,
 '68': 5,
 '67': 5,
 '66': 5,
 '65': 5,
 '64': 5,
 '63': 5,
 '62': 5,
 '61': 5,
 '60': 5,
 '59': 5,
 '58': 5,
 '57': 5,
 '56': 5,
 '55': 5,
 '54': 5,
 '53': 5,
 '52': 5,
 '51': 5,
 '50': 5,
 '49': 5,
 '48': 5,
 '47': 5,
 '46': 5,
 '45': 5,
 '44': 5,
 '43': 5,
 '42': 5,
 '41': 5,
 '40': 5,
 '39': 5,
 '38': 5,
 '37': 5,
 '36': 5,
 '35': 5,
 '34': 5,
 '33': 5,
 '32': 5,
 '31': 5,
 '30': 5,
 '29': 5,
 '28': 5,
 '27': 5,
 '26': 5,
 '25': 5,
 '24': 5,
 '23': 5,
 '22': 5,
 '21': 5,
 '20': 5,
 '19': 5,
 '18': 5,
 '17': 5,
 '16': 5,
 '15': 5,
 '14': 5,
 '13': 5,
 '12': 5,
 '11': 5,
 '10': 5,
 '09': 5,
 '08': 5,
 '07': 5,
 '06': 5,
 '05': 5,
 '04': 5,
 '03': 5,
 '02': 5,
 '01': 5}

In [40]:
count_males = 0 
count_females = 0 
male_list = []
for index, row in Crema_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1

In [41]:
count_males, count_females

(235, 220)

Since there are more males than females we will remove randomly 3 male actors (since there are exactly 5 audio files per actor)

In [42]:
import random 
random.seed(42)
males_to_remove = random.sample(male_list, 3)
males_to_remove

['17', '80', '88']

In [43]:
new_df = []
for index, row in Crema_df.iterrows(): 
    if row['actors'] not in males_to_remove: 
        new_df.append(row)

In [44]:
CREMA_df = pd.DataFrame(new_df)

In [45]:
for index, row in CREMA_df.iterrows(): 
    if row['actors'] == '17': 
        print("Elements not removed")

In [46]:
count_males = 0 
count_females = 0 
male_list = []
female_list = []
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1
        if actor not in female_list: 
            female_list.append(actor)

In [47]:
count_males, count_females

(220, 220)

In [48]:
len(female_list)

44

In [49]:
len(male_list)

44

In [50]:
CREMA_train = []
CREMA_val = []
CREMA_test = []

In [51]:
females_train = random.sample(female_list, 32)
males_train = random.sample(male_list, 32)

# remove the elements assigned to train 
for element in females_train:
    if element in female_list:
        female_list.remove(element)
        
for element in males_train:
    if element in male_list:
        male_list.remove(element)

         
females_val = random.sample(female_list, 6) 
males_val = random.sample(male_list, 6) 

# remove the elements assigned to val
for element in females_val:
    if element in female_list:
        female_list.remove(element)
        
for element in males_val:
    if element in male_list:
        male_list.remove(element)
        
females_test = random.sample(female_list, 6) 
males_test = random.sample(male_list, 6)        

In [52]:
females_train, males_train, females_val, males_val, females_test, males_test

(['54',
  '56',
  '58',
  '74',
  '76',
  '13',
  '78',
  '29',
  '84',
  '89',
  '09',
  '60',
  '04',
  '55',
  '52',
  '91',
  '02',
  '07',
  '46',
  '49',
  '37',
  '10',
  '20',
  '75',
  '21',
  '53',
  '06',
  '28',
  '18',
  '63',
  '30',
  '03'],
 ['57',
  '69',
  '65',
  '45',
  '77',
  '81',
  '41',
  '15',
  '44',
  '23',
  '59',
  '86',
  '34',
  '01',
  '85',
  '66',
  '31',
  '33',
  '05',
  '48',
  '50',
  '67',
  '51',
  '22',
  '36',
  '87',
  '71',
  '39',
  '42',
  '11',
  '32',
  '14'],
 ['43', '61', '40', '47', '73', '24'],
 ['62', '68', '64', '83', '70', '26'],
 ['08', '79', '12', '25', '72', '82'],
 ['16', '19', '38', '35', '27', '90'])

In [53]:
train = females_train + males_train 
val = females_val + males_val 
test = females_test + males_test

In [54]:
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if actor in train: 
        CREMA_train.append(row)
    elif actor in val: 
        CREMA_val.append(row)
    else:
        CREMA_test.append(row)

In [55]:
CREMA_train = pd.DataFrame(CREMA_train) 
CREMA_val = pd.DataFrame(CREMA_val) 
CREMA_test = pd.DataFrame(CREMA_test)

In [56]:
CREMA_train.shape, CREMA_val.shape, CREMA_test.shape

((320, 4), (60, 4), (60, 4))

In [57]:
CREMA_train = CREMA_train.reset_index(drop=True) 
CREMA_val = CREMA_val.reset_index(drop = True) 

# Random Search parameters

In [58]:
param_grid_clf = {'C': [0.1,1, 10, 100], 'kernel': ['rbf',  'linear']}
svc = SVC()

# Experiment 1.1 : RAVDESS

In [59]:
df_train = RAV_train
df_val = RAV_val
df_test = RAV_test

In [60]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [61]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26)

100%|████████████████████████████████████████| 120/120 [00:00<00:00, 796.41it/s]


In [62]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [63]:
np.size(y_val)

120

In [64]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [65]:
X_train.shape

(1200, 26)

In [66]:
len(X_train[0])

26

In [67]:
y_train.shape

(1200,)

## Shuffle training data

In [68]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [69]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [70]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [71]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 6.880s



In [72]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [73]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.0714285714285714, 1: 0.9375})

## Testing

In [74]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.65


## Save best model 

In [75]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_1.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [76]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.2 : RAVDESS noise

## Read dataframes

In [77]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/ravdess"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_val = pd.read_csv(os.path.join(preprocess_path,"df_val.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [78]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26) # 13

100%|████████████████████████████████████████| 120/120 [00:00<00:00, 793.22it/s]


In [79]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [80]:
np.size(y_val)

120

In [81]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [82]:
X_train.shape

(2400, 26)

## Shuffle training data

In [83]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [84]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [85]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [86]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 9.025s



In [87]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [88]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.0714285714285714, 1: 0.9375})

## Testing

In [89]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.7


## Save best model 

In [90]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_2.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [91]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.3: TESS

In [92]:
df_train = TESS_train
df_test = TESS_test

In [93]:
df_train.reset_index(drop = True, inplace = True) 
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [94]:
X_train, y_train, X_test, y_test = feature_extractor_tess(df_train,  df_test, 26)

100%|██████████████████████████████████████| 1400/1400 [00:02<00:00, 699.17it/s]


In [95]:
y_train, y_test = encode_labels_tess(y_train, y_test)

In [96]:
np.size(y_test)

1400

In [97]:
X_train, X_test = standard_scaling_tess(X_train,  X_test)

In [98]:
X_train.shape

(1400, 26)

## Shuffle training data

In [99]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [100]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [101]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [102]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 0.337s



In [103]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 1}

In [104]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=1, class_weight={0: 1.75, 1: 0.7})

## Testing

In [105]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.5714285714285714


## Save best model 

In [106]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_3.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [107]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.4: TESS noise

## Read dataframes

In [108]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/tess"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [109]:
X_train, y_train, X_test, y_test = feature_extractor_tess(df_train, df_test, 26) # 13

100%|██████████████████████████████████████| 1400/1400 [00:01<00:00, 895.95it/s]


In [110]:
y_train,  y_test = encode_labels_tess(y_train,  y_test)

In [111]:
np.size(y_train)

2800

In [112]:
X_train, X_test = standard_scaling_tess(X_train,  X_test)

In [113]:
X_train.shape

(2800, 26)

## Shuffle training data

In [114]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [115]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [116]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [117]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 4.918s



In [118]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 100}

In [119]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=100, class_weight={0: 1.75, 1: 0.7})

## Testing

In [120]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.5714285714285714


## Save best model 

In [121]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_4.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [122]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.5: SAVEE

In [123]:
df_train = SAVEE_train
df_val = SAVEE_val
df_test = SAVEE_test

In [124]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [125]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26)

100%|████████████████████████████████████████| 120/120 [00:00<00:00, 786.28it/s]


In [126]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [127]:
np.size(y_val)

120

In [128]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [129]:
X_train.shape

(240, 26)

## Shuffle training data

In [130]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [131]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [132]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [133]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 0.479s



In [134]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [135]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.0, 1: 1.0})

## Testing

In [136]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.5166666666666667


## Save best model 

In [137]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_5.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [138]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.6: SAVEE noise

## Read dataframes

In [139]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/savee"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_val = pd.read_csv(os.path.join(preprocess_path,"df_val.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [140]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26) # 13

100%|███████████████████████████████████████| 120/120 [00:00<00:00, 1043.02it/s]


In [141]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [142]:
np.size(y_val)

120

In [143]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [144]:
X_train.shape

(480, 26)

## Shuffle training data

In [145]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [146]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [147]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [148]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 1.270s



In [149]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [150]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.0, 1: 1.0})

## Testing

In [151]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.5


## Save best model 

In [152]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_6.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [153]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.7: CREMA 

In [154]:
df_train = CREMA_train
df_val = CREMA_val
df_test = CREMA_test

In [155]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [156]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26)

100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 915.93it/s]


In [157]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [158]:
np.size(y_val)

60

In [159]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [160]:
X_train.shape

(320, 26)

## Shuffle training data

In [161]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [162]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [163]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [164]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 0.490s



In [165]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 0.1}

In [166]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=0.1, class_weight={0: 2.5, 1: 0.625})

## Testing

In [167]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.4666666666666667


## Save best model 

In [168]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_7.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [169]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.8: CREMA - noise

In [170]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/crema"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_val = pd.read_csv(os.path.join(preprocess_path,"df_val.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [171]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26) # 13

100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 889.87it/s]


In [172]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [173]:
np.size(y_val)

60

In [174]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [175]:
X_train.shape

(640, 26)

## Hypeparameter optimization

In [176]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [177]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [178]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 5.035s



In [179]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [180]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 2.5, 1: 0.625})

## Testing

In [181]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.85


## Save best model 

In [182]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_8.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [183]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.9: RAVDESS - TESS - SAVEE

In [184]:
df_train = pd.concat([RAV_train, SAVEE_train, TESS_train])
df_val = pd.concat([RAV_val, SAVEE_val])
df_test = pd.concat([RAV_test, SAVEE_test, TESS_test])
#df_test = pd.concat([RAV_train, SAVEE_test])

In [185]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [186]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26)

100%|█████████████████████████████████████| 1640/1640 [00:01<00:00, 1594.54it/s]


In [187]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [188]:
np.size(y_test)

1640

In [189]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [190]:
X_train.shape

(2840, 26)

## Shuffle training data

In [191]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [192]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [193]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [194]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 10.528s



In [195]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [196]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.3148148148148149, 1: 0.8068181818181818})

## Testing

In [197]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.65


## Save best model 

In [198]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_9.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [199]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.10: RAVDESS - TESS - SAVEE noise

## Read dataframes

In [200]:
preprocess_path_rav = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/ravdess"
preprocess_path_savee = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/savee"
preprocess_path_tess = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/tess"

df_train_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_train.csv"))
df_val_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_val.csv"))
df_test_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_test.csv"))  

df_train_tess = pd.read_csv(os.path.join(preprocess_path_tess,"df_train.csv"))
df_test_tess= pd.read_csv(os.path.join(preprocess_path_tess,"df_test.csv"))  

df_train_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_train.csv"))
df_val_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_val.csv"))
df_test_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_test.csv"))  

In [201]:
df_train = pd.concat([df_train_rav, df_train_savee, df_train_tess])
df_val = pd.concat([df_val_rav, df_val_savee])
#df_test = pd.concat([df_test_rav, df_test_savee, df_test_tess])
df_test = pd.concat([RAV_train, SAVEE_test])

In [202]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [203]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26)

100%|█████████████████████████████████████| 1320/1320 [00:00<00:00, 1629.42it/s]


In [204]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [205]:
np.size(y_test)

1320

In [206]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [207]:
X_train.shape

(5680, 26)

## Shuffle training data

In [208]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [209]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [210]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [211]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 37.337s



In [212]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [213]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.3148148148148149, 1: 0.8068181818181818})

## Testing

In [214]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.878030303030303


## Save best model 

In [215]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_10.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [216]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.11: RAVDESS - TESS - SAVEE - CREMA

In [217]:
df_train = pd.concat([RAV_train, SAVEE_train, TESS_train, CREMA_train])
df_val = pd.concat([RAV_val, SAVEE_val, CREMA_val])
df_test = pd.concat([RAV_test, SAVEE_test, TESS_test, CREMA_test])
#df_test = pd.concat([RAV_train, SAVEE_test])

In [218]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [219]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26)

100%|█████████████████████████████████████| 1700/1700 [00:01<00:00, 1672.29it/s]


In [220]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [221]:
np.size(y_test)

1700

In [222]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [223]:
X_train.shape

(3160, 26)

## Shuffle training data

In [224]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [225]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [226]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [227]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 13.442s



In [228]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [229]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.381118881118881, 1: 0.7837301587301587})

## Testing

In [230]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.6594117647058824


## Save best model 

In [231]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_11.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [232]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"

# Experiment 1.12:  RAVDESS - TESS - SAVEE - CREMA noise

## Read dataframes

In [233]:
preprocess_path_rav = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/ravdess"
preprocess_path_savee = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/savee"
preprocess_path_tess = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/tess"
preprocess_path_crema = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/crema"

df_train_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_train.csv"))
df_val_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_val.csv"))
df_test_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_test.csv"))  

df_train_tess = pd.read_csv(os.path.join(preprocess_path_tess,"df_train.csv"))
df_test_tess= pd.read_csv(os.path.join(preprocess_path_tess,"df_test.csv"))  

df_train_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_train.csv"))
df_val_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_val.csv"))
df_test_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_test.csv"))  

df_train_crema = pd.read_csv(os.path.join(preprocess_path_crema,"df_train.csv"))
df_val_crema = pd.read_csv(os.path.join(preprocess_path_crema,"df_val.csv"))
df_test_crema = pd.read_csv(os.path.join(preprocess_path_crema,"df_test.csv")) 

In [234]:
df_train = pd.concat([df_train_rav, df_train_savee, df_train_tess, df_train_crema])
df_val = pd.concat([df_val_rav, df_val_savee, df_train_crema, df_train_crema])
#df_test = pd.concat([df_test_rav, df_test_savee, df_test_tess])
#df_test = pd.concat([df_test_rav, df_test_savee, df_test_crema ])
df_test = pd.concat([df_test_rav, df_test_savee ])

In [235]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [236]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 26) # 13

100%|███████████████████████████████████████| 240/240 [00:00<00:00, 1206.92it/s]


In [237]:
X_train[0].shape

(26,)

In [238]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [239]:
np.size(y_val)

1520

In [240]:
X_train, X_val, X_test = standard_scaling(X_train, X_val, X_test)

In [241]:
X_train.shape

(6320, 26)

## Shuffle training data

In [242]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Hypeparameter optimization

In [243]:
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [244]:
rand_search = RandomizedSearchCV(estimator=svc, 
                                 param_distributions=param_grid_clf, 
                                 n_jobs = -1, 
                                 cv=KFold(3))

In [245]:
print("Performing Randomized Search...")
t0 = time()
rand_search.fit(X_train, y_train) 
print("\nDone in %0.3fs" % (time()-t0))
print() 

Performing Randomized Search...

Done in 42.224s



In [246]:
best_params = rand_search.best_params_
best_clf = rand_search.best_estimator_
best_params

{'kernel': 'rbf', 'C': 10}

In [247]:
best_clf =  SVC(C=best_params.get('C'), kernel=best_params.get('kernel'), class_weight=class_weights)
best_clf.fit(X_train, y_train)

SVC(C=10, class_weight={0: 1.381118881118881, 1: 0.7837301587301587})

## Testing

In [248]:
print("Testing...")
pred = best_clf.predict(X_test) 
accuracy = best_clf.score(X_test, y_test) 
print("Accuracy: %s" %str(accuracy))

Testing...
Accuracy: 0.5625


## Save best model 

In [249]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_8/model_8_12.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_clf, file)

In [250]:
'''
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
'''

"\n# Load from file\nwith open(pkl_filename, 'rb') as file:\n    pickle_model = pickle.load(file)\n"